In [1]:
import os, io
import sqlite3
from sqlite3 import Error
import requests
import zipfile
import csv
import glob
import pandas as pd
import codecs

In [2]:
# Downloading the Medicare Hospital Compare Data

url = 'https://data.medicare.gov/views/bg9k-emty/files/0a9879e0-3312-4719-a1db-39fd114890f1? \
       content_type=application%2Fzip%3B%20charset%3Dbinary&filename=Hospital_Revised_Flatfiles.zip'
request = requests.get(url)
file = zipfile.ZipFile(io.BytesIO(request.content))
file.extractall('staging')

# UTF-8 conversion

files = glob.glob(os.path.join("staging" + "/*.csv"))

dict_ = {}
for filename in files:
    if filename == 'Hospital.pdf' or filename == 'readme.txt' or filename.endswith(
            'zip') or filename == 'FY2015_Percent_Change_in_Medicare_Payments.csv':
        print(filename)
    else:
        dict_[filename] = pd.read_csv(filename, header=0, encoding='cp1252').dropna(axis=1, how='all')

for file in dict_:
    dict_[file].to_csv(file, encoding='utf-8')

dir_path = os.path.abspath(os.path.realpath("staging"))
dir_path = dir_path

In [3]:
## Creating the Database

def create_connection(db_file):
    """ create a database connection to a SQLite database """
    try:
        con = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        con.close()


create_connection("medicare_hospital_compare_1.db")

2.6.0


In [4]:
# creating the tables from CSV


for filename in os.listdir(dir_path):

    if filename == 'Hospital.pdf' or filename == 'readme.txt' or filename.endswith(
            'zip') or filename == 'FY2015_Percent_Change_in_Medicare_Payments.csv':
        continue
    else:
        filepath = dir_path + "\\" + filename
        con = sqlite3.connect("medicare_hospital_compare_1.db")
        c = con.cursor()
        with open(filepath, 'rt', encoding="utf8") as fin:
            filename = filename.lower()
            filename = filename.replace("-", "_")
            filename = filename.replace(" ", "_")
            filename = filename.replace("/", "_")
            filename = filename.replace("%", "pct")
            filename = filename.replace(".csv", "")
            if filename[0].isalpha():
                filename = filename
            else:
                filename = "t_" + filename
            dr = csv.reader(fin)  # comma is default delimiter
            row1 = next(dr)
            column_count = len(list(list(dr)[0]))
            CR = "DROP TABLE IF EXISTS filename;"
            create_table_statement = "CREATE TABLE IF NOT EXISTS " + filename + " ("
            for i in range(0, column_count):
                row1[i] = row1[i].lower()
                row1[i] = row1[i].replace("-", "_")
                row1[i] = row1[i].replace(" ", "_")
                row1[i] = row1[i].replace("/", "_")
                row1[i] = row1[i].replace("%", "pct")

                if row1[i][:1].isalpha():
                    row1[i] = row1[i]
                else:
                    row1[i] = "c_" + row1[i]

                create_table_statement = create_table_statement + '{} text,'.format(row1[i])
            create_table_statement = create_table_statement[:-1] + ');'
            c.execute(CR)
            c.execute(create_table_statement)

            df = pd.read_csv(filepath, header=0)
            df.to_sql(filename, con, if_exists='replace', dtype="text", index=False)

            con.commit()
            c.close()


C:\Users\Rushabh Shah\Anaconda3\lib\site-packages\pandas\core\generic.py:1345: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [5]:
## Downloding hospital_ranking_focus_states


import requests
import openpyxl

url = "http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx"
r = requests.get(url)
r.headers
xf = open("hospital_ranking_focus_states.xlsx","wb")
xf.write(r.content)
xf.close()
wb = openpyxl.load_workbook("hospital_ranking_focus_states.xlsx")
    
filename="hospital_ranking_focus_states.xlsx" 
con=sqlite3.connect("medicare_hospital_compare_1.db")
wb=pd.read_excel(filename,sheetname=None)
for sheet in wb:
    wb[sheet].to_sql(sheet,con, index=False)
con.commit()
con.close()


C:\Users\Rushabh Shah\Anaconda3\lib\site-packages\pandas\core\generic.py:1345: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [6]:

##  Creating a hospital ranking MS Excel Workbook named “hospital_ranking.xlsx”

import openpyxl
wb = openpyxl.Workbook()
sheet_1 = wb.create_sheet("Nationwide")
sheet_1.cell(row = 1, column = 1, value = "Provider ID")
sheet_1.cell(row = 1, column = 2, value = "Hospital Name")
sheet_1.cell(row = 1, column = 3, value = "City")
sheet_1.cell(row = 1, column = 4, value = "State")
sheet_1.cell(row = 1, column = 5, value = "County")
wb.save("hospital_ranking.xlsx")


##importing data to hospital_ranking excel file

con = sqlite3.connect("medicare_hospital_compare_1.db")
c = con.cursor()
selecting_data = "select g.[Provider ID], g.[Hospital Name], g.City, g.State, g.[County Name] as 'County' from hospital_general_information as g inner join [Hospital National Ranking] as h on g.[Provider ID] = h.[Provider ID] order by h.Ranking limit 100;"
df = pd.read_sql(selecting_data, con)

writer = pd.ExcelWriter('hospital_ranking.xlsx')
df.to_excel(writer, sheet_name ='Nationwide', index= False)
####writer.save()

con.close()


In [7]:
## Creating the state wise sheet 

wb = openpyxl.load_workbook("hospital_ranking_focus_states.xlsx")
sheet_2_name = wb.get_sheet_names()[1]
sheet_2 = wb.get_sheet_by_name(sheet_2_name)
i = 2
state=[]
abstate=[]
while sheet_2.cell(row = i, column = 1).value != None:
    state.append(sheet_2.cell(row = i,column = 1).value)
    abstate.append(sheet_2.cell(row = i, column = 2).value)
    i += 1
state.sort()
abstate.sort()
abstate
wb.close()

con = sqlite3.connect("medicare_hospital_compare_1.db")
c = con.cursor()


In [8]:
##Creating the state wise sheet


wb = openpyxl.load_workbook("hospital_ranking.xlsx")
for states in state: 
    sheet_1 = wb.create_sheet(states)
    sheet_1.cell(row = 1, column = 1, value = "Provider ID")
    sheet_1.cell(row = 1, column = 2, value = "Hospital Name")
    sheet_1.cell(row = 1, column = 3, value = "City")
    sheet_1.cell(row = 1, column = 4, value = "State")
    sheet_1.cell(row = 1, column = 5, value = "County")
wb.save("hospital_ranking.xlsx")
    
   

##Adding the data to newly state sheet
count = 0     
#con = sqlite3.connect("medicare_hospital_compare_1.db")
#c = con.cursor()
####writer = pd.ExcelWriter('hospital_ranking.xlsx')
for states in state:
    
    selecting_data = "select g.[Provider ID], g.[Hospital Name], g.City, g.State, g.[County Name] as 'County' from hospital_general_information as g inner join [Hospital National Ranking] as h on g.[Provider ID] = h.[Provider ID] where g.State= " + "'" + abstate[count] + "'" + " order by h.Ranking limit 100;"
    df = pd.read_sql(selecting_data, con)
    
    df.to_excel(writer, sheet_name = states, index=False)
    count = count + 1

writer.save()
writer.close()
wb.close() 

In [9]:
#Creating the Measure

data="select State, [Measure ID], [Measure Name], Score from  timely_and_effective_care___hospital  where length(Score) < 7 order by [Measure ID];"
df = pd.read_sql(data, con)

In [10]:
df['Score']= df['Score'].astype(float)

In [11]:
group = df.groupby('Measure ID')

In [12]:
group.min()
group.max()
group.mean()
group.std()

,Score
Measure ID,
ED_1b,103.307105
ED_2b,64.940604
IMM_2,13.170603
IMM_3_OP_27_FAC_ADHPCT,14.878308
OP_1,6.737422
OP_18b,43.567914
OP_2,21.479484
OP_20,18.346156
OP_21,18.477840


In [13]:
import numpy as np
df_statistics = (group.agg([np.min, np.max, np.mean, np.std]).rename(columns={'amin':'Minimum','amax': 'Maximum','mean': 'Average','std': 'Standard Deviation'}))
statistics = df_statistics.copy()

In [14]:
data = []
for i in range (0,len(statistics)):
    data.append(statistics.iloc[i].reset_index(drop = True))

In [15]:
m_stat = pd.DataFrame(data)
m_stat.reset_index(inplace = True)
m_stat.drop('index', axis =1)
m_stat.columns = ['Measure ID','Minimum','Maximum','Average','Standard Deviation']

In [16]:
m_stat['Measure Name'] = m_stat['Measure ID']

In [17]:
final_stat = m_stat[['Measure ID','Measure Name','Minimum', 'Maximum','Average', 'Standard Deviation']]

In [18]:

excel_writer = pd.ExcelWriter('measure_statistics.xlsx', engine = 'xlsxwriter')

final_stat.to_excel(excel_writer, sheet_name = 'Nationwide', index = False)


In [19]:
## Creating state wise measure
count=0
for states in state:
    data="select State, [Measure ID], [Measure Name], Score from  timely_and_effective_care___hospital  where length(Score) < 7 and State= " + "'" + abstate[count] + "'" + "order by [Measure ID];"
    df = pd.read_sql(data, con)
    df['Score']= df['Score'].astype(float)
    group = df.groupby('Measure ID')
    df_statistics = (group.agg([np.min, np.max, np.mean, np.std]).rename(columns={'amin':'Minimum','amax': 'Maximum','mean': 'Average','std': 'Standard Deviation'}))
    statistics = df_statistics.copy()
    data = []
    for i in range (0,len(statistics)):
        data.append(statistics.iloc[i].reset_index(drop = True))
    m_stat = pd.DataFrame(data)
    m_stat.reset_index(inplace = True)
    m_stat.drop('index', axis =1)
    m_stat.columns = ['Measure ID','Minimum','Maximum','Average','Standard Deviation']
    m_stat['Measure Name'] = m_stat['Measure ID']
    final_stat = m_stat[['Measure ID','Measure Name','Minimum', 'Maximum','Average', 'Standard Deviation']]
    #final_stat.to_excel()
    final_stat.to_excel(excel_writer, sheet_name = states, index = False)
    count = count + 1
excel_writer.save()   